In [3]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import time

key = RSA.generate(2048)
private_key = key.export_key()
public_key = key.publickey().export_key()


cipher_rsa = PKCS1_OAEP.new(RSA.import_key(public_key))
decipher_rsa = PKCS1_OAEP.new(RSA.import_key(private_key))

# Зашифрование сообщения
start_time = time.time()
encrypted_message = cipher_rsa.encrypt(b'hamsterGlavnyi123')
print("Время шифрования RSA: ", time.time() - start_time)

# Расшифровка сообщения
start_time = time.time()
decrypted_message = decipher_rsa.decrypt(encrypted_message)
print("Время дешифрования RSA: ", time.time() - start_time)


Время шифрования RSA:  0.0038461685180664062
Время дешифрования RSA:  0.011894464492797852


In [4]:
from elgamal.elgamal import Elgamal
import time

# Генерация ключей Эль-Гамаля
pb, pv = Elgamal.newkeys(128)

# Зашифрование сообщения
start_time = time.time()
encrypted_message = Elgamal.encrypt('hamsterGlavnyi123'.encode(), pb)
print("Время шифрования Эль-Гамаля: ", time.time() - start_time)

# Расшифровка сообщения
start_time = time.time()
decrypted_message = Elgamal.decrypt(encrypted_message, pv)
print("Время дешифрования Эль-Гамаля: ", time.time() - start_time)


Время шифрования Эль-Гамаля:  0.14873576164245605
Время дешифрования Эль-Гамаля:  0.10336542129516602


In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import matplotlib.pyplot as plt
import time
import random
import secrets
import string
import base64


def gcd(a, b):
    if a < b:
        return gcd(b, a)
    elif a % b == 0:
        return b
    else:
        return gcd(b, a % b)


def gen_key(q):
    key = secrets.randbelow(q)
    while gcd(q, key) != 1:
        key = secrets.randbelow(q)
    return key


def power(a, b, c):
    x = 1
    y = a
    while b > 0:
        if b % 2 == 0:
            x = (x * y) % c
        y = (y * y) % c
        b = int(b / 2)
    return x % c


def encryption(msg, q, h, g):
    ct = []
    k = gen_key(q)
    s = power(h, k, q)
    p = power(g, k, q)
    for i in range(0, len(msg)):
        ct.append(msg[i])
    for i in range(0, len(ct)):
        ct[i] = s * ord(ct[i])
    return ct, p


def decryption(ct, p, key, q):
    pt = []
    h = power(p, key, q)
    for i in range(0, len(ct)):
        pt.append(chr(int(ct[i] / h)))
    return pt


def encode_base64(data):
    return base64.b64encode(data).decode()  # Преобразуем байтовую строку в строку и затем кодируем в base64

def decode_base64(encoded_data):
    return base64.b64decode(encoded_data.encode())


class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("ЛР10")
        self.geometry("600x300")
        

        self.style = ttk.Style(self)
# self.style.theme_use("clam") 

# Левая часть окна (кнопки и радиокнопки)
        left_frame = tk.Frame(self)
        left_frame.pack(side=tk.LEFT, padx=20, pady=20)

        self.encrypt_button = ttk.Button(left_frame, text="Шифрование", command=self.encrypt)
        self.encrypt_button.pack(pady=10)

        self.decrypt_button = ttk.Button(left_frame, text="Дешифрование", command=self.decrypt)
        self.decrypt_button.pack(pady=10)

        self.mode_var = tk.StringVar(value="ElGamal")
        rsa_radio = ttk.Radiobutton(left_frame, text="RSA", variable=self.mode_var, value="RSA")
        elgamal_radio = ttk.Radiobutton(left_frame, text="Эль-Гамаль", variable=self.mode_var, value="ElGamal")
        rsa_radio.pack(pady=5)
        elgamal_radio.pack(pady=5)

        self.base64_var = tk.BooleanVar(value=False)
        self.base64_checkbox = ttk.Checkbutton(left_frame, text="Base64", variable=self.base64_var)
        self.base64_checkbox.pack(pady=5)

        self.generate_graphs_button = ttk.Button(left_frame, text="Создать графики", command=self.generate_graphs)
        self.generate_graphs_button.pack(pady=10)

# Правая часть окна (текстовое поле)
        right_frame = tk.Frame(self)
        right_frame.pack(side=tk.LEFT, padx=20, pady=20)

        self.text_area_label = ttk.Label(right_frame, text="Введите сообщение")
        self.text_area_label.pack(pady=20)

        self.text_area = tk.Text(right_frame, height=10, width=60)
        self.text_area.pack(pady=5)

        self.rsa_key = RSA.generate(2048)

        # Параметры для Эльгамаля
        self.q = secrets.randbelow(int(10 ** 50))
        self.g = random.randint(2, self.q)
        self.key = gen_key(self.q)
        self.h = power(self.g, self.key, self.q)

    def encrypt(self):
        mode = self.mode_var.get()
        plaintext = self.text_area.get("1.0", "end-1c")
        use_base64 = self.base64_var.get()

        if mode == "RSA":
            self.handle_rsa(plaintext, use_base64)
        else:
            self.handle_elgamal(plaintext, use_base64)

    def handle_rsa(self, plaintext, use_base64):
        try:
            start_time = time.time()
            cipher = PKCS1_OAEP.new(self.rsa_key.publickey())
            ciphertext = cipher.encrypt(plaintext.encode())
            if use_base64:  # Проверяем, используется ли base64
                ciphertext = encode_base64(ciphertext)  # Кодируем шифртекст в base64
            else:
                ciphertext = ciphertext.hex()
            self.text_area.delete("1.0", "end")
            self.text_area.insert("1.0", ciphertext)
            elapsed_time = time.time() - start_time
            original_length = len(plaintext)
            encrypted_length = len(ciphertext)
            messagebox.showinfo("Результаты",
                                f"Время зашифр.: {elapsed_time:.6f} секунд\nДлина ориг. текста: {original_length} байт\nДлина зашифр. текста: {encrypted_length} байт")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    def handle_elgamal(self, plaintext, use_base64):
        try:
            start_time = time.time()
            ciphertext, p = encryption(plaintext, self.q, self.h, self.g)
            if use_base64:
                ciphertext_str = encode_base64(str(ciphertext))
                p_str = encode_base64(str(p))
                ciphertext_data = (ciphertext_str, p_str)
            else:
                ciphertext_data = (ciphertext, p)
            self.text_area.delete("1.0", "end")
            self.text_area.insert("1.0", str(ciphertext_data))
            elapsed_time = time.time() - start_time
            original_length = len(plaintext)
            encrypted_length = len(str(ciphertext_data))
            messagebox.showinfo("Результаты",
                                f"Время зашифр.: {elapsed_time:.6f} секунд\nДлина ориг. текста: {original_length} байт\nДлина зашифр. текста: {encrypted_length} байт")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    def decrypt(self):
        try:
            mode = self.mode_var.get()
            ciphertext = self.text_area.get("1.0", "end-1c")
            use_base64 = self.base64_var.get()

            if mode == "RSA":
                self.handle_rsa_decrypt(ciphertext, use_base64)
            else:
                self.handle_elgamal_decrypt(ciphertext, use_base64)
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    def handle_rsa_decrypt(self, ciphertext, use_base64):
        try:
            start_time = time.time()
            if use_base64:
                ciphertext = decode_base64(ciphertext)
            else:
                ciphertext = bytes.fromhex(ciphertext)
            cipher = PKCS1_OAEP.new(self.rsa_key)
            decrypted_text = cipher.decrypt(ciphertext)
            self.text_area.delete("1.0", "end")
            self.text_area.insert("1.0", decrypted_text.decode())
            elapsed_time = time.time() - start_time
            original_length = len(decrypted_text)
            encrypted_length = len(ciphertext)
            messagebox.showinfo("Результаты",
                                f"Время расшифр.: {elapsed_time:.6f} секунд\nДлина ориг. текста: {original_length} байт\nДлина зашифр. текста: {encrypted_length} байт")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    def handle_elgamal_decrypt(self, ciphertext, use_base64):
        try:
            start_time = time.time()
            if use_base64:
                ciphertext_data = eval(ciphertext)
                ciphertext_str, p_str = ciphertext_data
                ciphertext = eval(decode_base64(ciphertext_str))
                p = eval(decode_base64(p_str))
            else:
                ciphertext_data = eval(ciphertext)
                ciphertext, p = ciphertext_data
            plaintext = decryption(ciphertext, p, self.key, self.q)
            plaintext_str = ''.join(plaintext)
            self.text_area.delete("1.0", "end")
            self.text_area.insert("1.0", plaintext_str)
            elapsed_time = time.time() - start_time
            original_length = len(plaintext_str)
            encrypted_length = len(ciphertext)
            messagebox.showinfo("Результаты",
                                f"Время расшифр.: {elapsed_time:.6f} секунд\nДлина ориг. текста: {original_length} байт\nДлина зашифр. текста: {encrypted_length} байт")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    def generate_graphs(self):
        message_lengths = [13, 25, 50, 100, 300, 500, 1000]
        rsa_encrypt_times = []
        rsa_encrypted_lengths = []
        elgamal_encrypt_times = []
        elgamal_encrypted_lengths = []

        for length in message_lengths:
            random_text = ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

            def split_message(message, chunk_size):
                return [message[i:i + chunk_size] for i in range(0, len(message), chunk_size)]

            start_time = time.time()
            cipher = PKCS1_OAEP.new(self.rsa_key.publickey())
            chunk_size = int((self.rsa_key.size_in_bytes() - 11) / 8)
            rsa_ciphertexts = []
            for chunk in split_message(random_text.encode(), chunk_size):
                rsa_ciphertexts.append(cipher.encrypt(chunk))
            rsa_encrypt_times.append(time.time() - start_time)
            rsa_encrypted_lengths.append(len(rsa_ciphertexts))

            start_time = time.time()
            elgamal_ciphertext, p = encryption(random_text, self.q, self.h, self.g)
            elgamal_encrypt_times.append(time.time() - start_time)
            elgamal_encrypted_lengths.append(len(str((elgamal_ciphertext, p))))

    # График времени шифрования
        plt.figure(figsize=(10, 5))
        plt.plot(message_lengths, rsa_encrypt_times, label='RSA', color='green')
        plt.plot(message_lengths, elgamal_encrypt_times, label='Эль-Гамаль', color='orange')
        plt.title('Зависимость y от длины сообщения')
        plt.xlabel('Длина сообщения')
        plt.ylabel('Время шифрования')
        plt.legend()
        plt.show()

    # График длины зашифрованного текста
        # plt.figure(figsize=(10, 5))
        # plt.plot(message_lengths, rsa_encrypted_lengths, label='RSA', color='blue')
        # plt.plot(message_lengths, elgamal_encrypted_lengths, label='Эль-Гамаль', color='orange')
        # plt.title('Длина зашифрованного текста от длины сообщения')
        # plt.xlabel('Длина сообщения')
        # plt.ylabel('Длина зашифрованного текста')
        # plt.legend()
        # plt.show()


if __name__ == "__main__":
    app = App()
    app.mainloop()